## Preparing Data


In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('data/data.xlsx')

/home/chatrin/.local/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [17]:
df.head().T

,0,1,2,3,4
yyyy,1947.000000,1948.000000,1949.000000,1950.000000,1951.000000
Index,15.300000,15.200000,16.760000,20.410000,23.770000
D12,0.840000,0.930000,1.140000,1.470000,1.410000
E12,1.610000,2.290000,2.320000,2.840000,2.440000
b/m,0.725326,0.840948,0.796429,0.722538,0.721316
tbl,0.009500,0.011600,0.011000,0.013400,0.017300
AAA,0.028600,0.027900,0.025800,0.026700,0.030100
BAA,0.035200,0.035300,0.033100,0.032000,0.036100
lty,0.024300,0.023700,0.020900,0.022400,0.026900
cay,0.079056,0.054523,0.051693,0.004356,0.004147


In [24]:
# Our Target Equity Premium
df["erp"] = df["CRSP_SPvw"] - df["Rfree"]

# Data preparing
df['lag_idx'] = df['Index'].shift(1)


# Features
df['dp'] = np.log(df['D12']/df['Index'])
df["dy"] = np.log(df["D12"] / (df["lag_idx"]))
df["ep"] = np.log(df["E12"] / df["Index"])
df['de'] = np.log(df['D12']/df['E12'])
df['tms'] = df['lty'] - df['tbl']
df['dfy'] = df['BAA'] - df['AAA']
df['dfr'] = df['lty'] - df['corpr']

# Cleaning data
df = df.dropna()

## Problem
The full sample period is 1947-2024. The initial estimation window is 1947-1966. The out-of-sample
period covers 1967-2024. The whole raw data from Goyal and Welch is in the attached EXCEL file
(PredictorData2024.xlsx).

(a) Use the historical mean model to calculate the 1-step forecasts, forecast errors and MSE for the out-of-sample period.

(b) Use AR(1) model to calculate the 1-step forecasts, forecast errors and MSE for the out-of-sample period.

(c) Use each single predictor in the list above to predict the equity premium, calculate the 1-step forecasts, forecast errors and MSEs for the out-of-sample period. (hint: you need to do 16 predictive regression each with a single predictor in the right hand side of the regression)

(d) Use all the predictors in one multivariate regression to predict the equity premium, calculate the 1-step forecasts, forecast errors and MSEs for the out-of-sample period.

(e) Equally weight the 16 forecasts from (c) to calculate the combined forecast. Calculate the forecast errors and MSE from this combined forecast for the out-of-sample period.

(f) Compare the out-of-sample MSE of the forecasts in (a), (b), (c) , (d), and (e), and discuss which model is the best if we use the out-of-sample MSE to evaluate the predictive performance for the out-of-sample period.

(g) Use the best model chosen in (f) to forecast the annual equity premium in 2025

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

oos_years = range(1967, 2025)
forecasts = []
actuals = []

In [25]:
# TODO: a - d
for t in oos_years:
  train_mask = df['yyyy'] < t
  test_mask = df['yyyy'] == t
  
  x_cols = ['dp','dy','ep','de','svar','b/m','ntis','eqis','tbl','lty','ltr','tms','dfy','dfr','infl','ik']
  x_train = df.loc[train_mask, x_cols]
  print(x_train.head().T)
  y_train = df.loc[train_mask, 'erp']
  
  model = LinearRegression()
  model.fit(x_train, y_train)
  x_test = df.loc[test_mask, x_cols]
  pred = model.predict(x_test)[0]
  forecasts.append(pred)
  actual = df.loc[test_mask, 'erp'].values[0]
  actuals.append(actual)
  
mse = mean_squared_error(actuals, forecasts)
print(f'Mean Squared Error: {mse}')
  

             1         2         3         4         5
dp   -2.793866 -2.687967 -2.630763 -2.824835 -2.936193
dy   -2.800424 -2.590267 -2.433733 -2.672435 -2.824835
ep   -1.892744 -1.977428 -1.972221 -2.276426 -2.404314
de   -0.901123 -0.710539 -0.658542 -0.548408 -0.531879
svar  0.022100  0.012306  0.021461  0.012182  0.006590
b/m   0.840948  0.796429  0.722538  0.721316  0.694073
ntis  0.015015  0.027176  0.031359  0.036151  0.026536
eqis  0.157140  0.191869  0.227023  0.264824  0.202497
tbl   0.011600  0.011000  0.013400  0.017300  0.020900
lty   0.023700  0.020900  0.022400  0.026900  0.027900
ltr   0.033850  0.064391  0.000505 -0.039422  0.011693
tms   0.012100  0.009900  0.009000  0.009600  0.007000
dfy   0.007400  0.007300  0.005300  0.006000  0.005400
dfr  -0.017659 -0.012188  0.001248  0.053803 -0.007309
infl  0.029915 -0.020747  0.059322  0.060000  0.007547
ik    0.034930  0.029374  0.034471  0.032664  0.032688
             1         2         3         4         5
dp   -2.79